# Clasificación de Texto

Este notebook presenta la creación de un dataset de oraciones con su respectivo autor, con el que luego se entrenan varios clasificadores usando variaciones de redes feed-forward y distintos embeddings pre-entrenados.

## 0. Importación de Librerías

In [1]:
# Se importan las librerías necesarias para el desarrollo del proyecto
import re

import gensim
from gensim.parsing.preprocessing import STOPWORDS
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')


import numpy as np

## 1. Creación del Dataset

Primero, creamos el dataset de oraciones etiquetadas según el autor.

In [2]:
# Se define una función que carga un archivo de texto y lo devuelve como un string
def load_raw_data(file_path: str) -> str:
    """
    Carga el texto crudo a partir de un archivo de texto
    
    Args:
    file_path (str): Ruta del archivo de texto.
    
    Returns:
    str: Texto crudo.
    """
    # Leer el texto crudo
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    return text

In [3]:
# Se define una función que extrae las oraciones del texto según el formato de los libros de Gutenberg
def extract_sentences(book: str) -> list[str]:
    """
    Extrae extractos de un libro asegurando que cumplan con ciertas condiciones de tamaño
    
    Args:
    book (str): Texto crudo.
    
    Returns:
    list[str]: Lista de extractos del libro.
    """
    # Separar el texto en bloques usando líneas completamente vacías como delimitadores
    # Seleccionar el tercer bloque que contiene el contenido del libro
    lines = book.split('***')[2].split('\n\n')

    # Eliminar espacios en blanco al inicio y al final de cada línea
    lines = [line.strip() for line in lines]

    # Eliminar lineas vacias
    lines = [line for line in lines if line]

    # Eliminar lineas genericas como ilustraciones o titulos de los capitulos de los libros
    lines = [line for line in lines if not (
        line.startswith('CHAPTER')) or 
        line.startswith('[Illustration]')
    ]

    # Eliminar saltos de lineas de las oraciones
    lines = [line.replace('\n', ' ') for line in lines]

    # Solo procesar lineas con mas de 150 caracteres
    lines = [line for line in lines if len(line) >= 150]

    # Separar adicionalmente por . si la oracion es muy larga
    sentences = []
    for sentence in lines:
        if len(sentence) > 250:
            sentences.extend(sent_tokenize(sentence))    # Dividir en oraciones usando NLTK
        else:
            sentences.append(sentence)

    # Solo procesar lineas con mas de 150 y menos de 250 caracteres
    sentences = [sentence for sentence in sentences if len(sentence) >= 150 and len(sentence) <= 250]

    # Eliminar espacios en blanco al inicio y al final de cada línea nuevamente
    sentences = [sentence.strip() for sentence in sentences]

    return sentences

In [4]:
# Ruta a los libros originales junto con su autor
raw_books = {
    'austen_sense-and-sensibility': {
        'file_path': 'data/raw/austen_sense-and-sensibility.txt',
        'author': 'Jane Austen',
    },
    'austen_pride-and-prejudice': {
        'file_path': 'data/raw/austen_pride-and-prejudice.txt',
        'author': 'Jane Austen',
    },
    'austen_emma': {
        'file_path': 'data/raw/austen_emma.txt',
        'author': 'Jane Austen',
    },
    'tolstoy_youth': {
        'file_path': 'data/raw/tolstoy_youth.txt',
        'author': 'Leo Tolstoy',
    },
    'tolstoy_war-and-peace': {
        'file_path': 'data/raw/tolstoy_war-and-peace.txt',
        'author': 'Leo Tolstoy',
    },
    'tolstoy_anna-karenina': {
        'file_path': 'data/raw/tolstoy_anna-karenina.txt',
        'author': 'Leo Tolstoy',
    },
    'joyce_dubliners': {
        'file_path': 'data/raw/joyce_dubliners.txt',
        'author': 'James Joyce',
    },
    'joyce_a-portrait-of-the-artist-as-a-young-man': {
        'file_path': 'data/raw/joyce_a-portrait-of-the-artist-as-a-young-man.txt',
        'author': 'James Joyce',
    },
    'joyce_ulysses': {
        'file_path': 'data/raw/joyce_ulysses.txt',
        'author': 'James Joyce',
    }
}

In [5]:
# Se crea un dataframe con las oraciones extraídas de los libros
df = pd.DataFrame(columns=['author', 'sentence'])

# Por cada libro, se extra el texto y se concatenan las oraciones en el dataframe
for book in raw_books.values():
    corpus = load_raw_data(book['file_path'])
    author = book['author']
    
    # Extraer las oraciones del texto
    sentences = extract_sentences(corpus)

    df = pd.concat([df, pd.DataFrame({'author': author, 'sentence': sentences})], ignore_index=True)

df.head()

,author,sentence
0,Jane Austen,"Their estate was large, and their residence wa..."
1,Jane Austen,The late owner of this estate was a single man...
2,Jane Austen,"The son, a steady respectable young man, was a..."
3,Jane Austen,To him therefore the succession to the Norland...
4,Jane Austen,"Their mother had nothing, and their father onl..."


In [6]:
# Guardamos el dataset como un archivo CSV
df.to_csv('data/classifier/sentences.csv', index=False)

In [7]:
# Contar el número de datos de entrenamiento por cada autor
author_counts = df['author'].value_counts()

# Convertir los conteos en un DataFrame
summary_df = author_counts.reset_index()
summary_df.columns = ['author', 'num_training_data']

summary_df

,author,num_training_data
0,Leo Tolstoy,10514
1,Jane Austen,3745
2,James Joyce,2815


## 2. Preprocesamiento del Dataset

Preprocesamos el dataset separandolo en entrenamiento y prueba. Adicionalmente, tokenizamos el texto para poder mapear las palabras a los embeddings construidos y usarlos como la capa de entrada de los modelos de redes neuronales.

In [8]:
# Conjunto de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(df['sentence'], df['author'],
                                                    train_size=0.7, random_state=42)

In [9]:
# Tokenización usando Keras
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

# Convertir el texto en secuencias de enteros
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Rellenar las secuencias para que tengan la misma longitud
max_length = max([len(seq) for seq in x_train_seq])
x_train_pad = pad_sequences(x_train_seq, maxlen=max_length, padding='post')
x_test_pad = pad_sequences(x_test_seq, maxlen=max_length, padding='post')

## 3. Definición de los Modelos de Redes Neuronales

Cargamos los los embeddings de Word2Vec pre-entrenados, creamos las capas de embeddings a partir de ellos, y definimos los tres tipos de arquitecturas de redes neuronales que usaremos.

In [10]:
# Ruta a los modelos Word2Vec combinados con diferentes tamaños de vectores
books_models = [
    'data/models/Books_50_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba.model',
    'data/models/Books_100_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba.model',
    'data/models/Books_300_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba.model'
]

# Cargar los embeddings de Word2Vec pre-entrenados
word2vec_model_50 = gensim.models.Word2Vec.load(books_models[0])
word2vec_model_100 = gensim.models.Word2Vec.load(books_models[1])
word2vec_model_300 = gensim.models.Word2Vec.load(books_models[2])

In [11]:
def create_embedding_layer(word2vec_model, tokenizer, max_length):
    """
    Crea una capa de embeddings a partir de un modelo Word2Vec y un tokenizer.

    Args:
    word2vec_model: Modelo Word2Vec preentrenado.
    tokenizer: Tokenizer que contiene el índice de palabras.
    max_length (int): Longitud máxima de las secuencias de entrada.

    Returns:
    Embedding: Capa de embedding de Keras que utiliza la matriz de embeddings generada.
    """
    # Crear la matriz de embeddings para el modelo Word2Vec
    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, word2vec_model.vector_size))
    for word, i in tokenizer.word_index.items():
        if word in word2vec_model.wv:
            embedding_matrix[i] = word2vec_model.wv[word]

    # Definir la capa de embedding en Keras
    embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1,
                                output_dim=word2vec_model.vector_size,
                                weights=[embedding_matrix],
                                input_length=max_length,
                                trainable=False)
    
    return embedding_layer

# Crear las capas de embeddings a partir de los modelos Word2Vec
embedding_layer_50 = create_embedding_layer(word2vec_model_50, tokenizer, max_length)
embedding_layer_100 = create_embedding_layer(word2vec_model_100, tokenizer, max_length)
embedding_layer_300 = create_embedding_layer(word2vec_model_300, tokenizer, max_length)

In [12]:
# Arquitectura 1: Modelo sencillo
def create_ffnn_model_1(embedding_layer):
    """
    Crea un modelo de red neuronal feedforward simple.

    Args:
    embedding_layer: Capa de embeddings de Keras utilizada como entrada.

    Returns:
    Sequential: Modelo de red neuronal compilado.
    """
    model = Sequential()
    model.add(embedding_layer)
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # Salida con 3 clases (autores)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Arquitectura 2: Modelo con más capas
def create_ffnn_model_2(embedding_layer):
    """
    Crea un modelo de red neuronal feedforward con más capas.

    Args:
    embedding_layer: Capa de embeddings de Keras utilizada como entrada.

    Returns:
    Sequential: Modelo de red neuronal compilado.
    """
    model = Sequential()
    model.add(embedding_layer)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Arquitectura 3: Modelo con más unidades
def create_ffnn_model_3(embedding_layer):
    """
    Crea un modelo de red neuronal feedforward con más unidades.

    Args:
    embedding_layer: Capa de embeddings de Keras utilizada como entrada.

    Returns:
    Sequential: Modelo de red neuronal compilado.
    """
    model = Sequential()
    model.add(embedding_layer)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## 4. Creación y Evaluación de los Modelos de Redes Neuronales

Creamos un modelo con cada tipo de arquitectura y capa de embeddings y evaluamos su accuracy, precision y recall.

In [13]:
# Codificación de las etiquetas (es decir, los autores)
label_encoder = LabelEncoder()
y_train_encoded = to_categorical(label_encoder.fit_transform(y_train))
y_test_encoded = to_categorical(label_encoder.transform(y_test))

In [14]:
def evaluate_model(model, x_test_pad, y_test_encoded):
    """
    Evalúa el rendimiento de un modelo entrenado calculando accuracy, precision y recall.
    
    Args:
    model (keras.models.Model): El modelo entrenado.
    x_test_pad (numpy.ndarray): Conjunto de datos de prueba preprocesados y tokenizados.
    y_test_encoded (numpy.ndarray): Etiquetas de prueba codificadas en formato one-hot.
    
    Returns:
    tuple: Un tupla que contiene:
        - accuracy (float): La proporción de predicciones correctas.
        - precision (float): La proporción de predicciones positivas correctas (precisión macro).
        - recall (float): La proporción de verdaderos positivos detectados (recall macro).
    """
    # Obtener predicciones del modelo
    y_pred = model.predict(x_test_pad)
    
    # Convertir las predicciones y etiquetas de one-hot a clases
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test_encoded, axis=1)
    
    # Calcular accuracy
    accuracy = np.mean(y_pred_classes == y_test_classes)
    
    # Calcular precisión y recall usando la métrica macro (promedio entre todas las clases)
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    
    return accuracy, precision, recall

### Modelo 1

In [15]:
# Instanciar el modelo 1 con los embeddings de 50 dimensiones
model_1_50 = create_ffnn_model_1(embedding_layer_50)
model_1_50.summary()

# Entrenar el modelo 1 con los embeddings de 50 dimensiones
model_1_50.fit(x_train_pad, y_train_encoded, epochs=5, batch_size=32, validation_data=(x_test_pad, y_test_encoded))

# Evaluar el modelo 1 con los embeddings de 50 dimensiones
accuracy, precision, recall = evaluate_model(model_1_50, x_test_pad, y_test_encoded)
print(f"Modelo 1 con 50 dimensiones - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 55, 50)            1175900   
                                                                 
 flatten (Flatten)           (None, 2750)              0         
                                                                 
 dense (Dense)               (None, 128)               352128    
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 1,528,415
Trainable params: 352,515
Non-trainable params: 1,175,900
_________________________________________________________________
Epoch 1/5
374/374 [==============================] - 5s 8ms/step - loss: 0.4754 - accuracy: 0.8038 - val_loss: 0.3986 - val_accuracy: 0.8407
Epoch 2/5
374/374 [============

In [16]:
# Instanciar el modelo 1 con los embeddings de 100 dimensiones
model_1_100 = create_ffnn_model_1(embedding_layer_100)
model_1_100.summary()

# Entrenar el modelo 1 con los embeddings de 100 dimensiones
model_1_100.fit(x_train_pad, y_train_encoded, epochs=5, batch_size=32, validation_data=(x_test_pad, y_test_encoded))

# Evaluar el modelo 1 con los embeddings de 100 dimensiones
accuracy, precision, recall = evaluate_model(model_1_100, x_test_pad, y_test_encoded)
print(f"Modelo 1 con 100 dimensiones - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 55, 100)           2351800   
                                                                 
 flatten_1 (Flatten)         (None, 5500)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               704128    
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 3,056,315
Trainable params: 704,515
Non-trainable params: 2,351,800
_________________________________________________________________
Epoch 1/5
374/374 [==============================] - 5s 10ms/step - loss: 0.4623 - accuracy: 0.8101 - val_loss: 0.3878 - val_accuracy: 0.8421
Epoch 2/5
374/374 [=========

In [17]:
# Instanciar el modelo 1 con los embeddings de 300 dimensiones
model_1_300 = create_ffnn_model_1(embedding_layer_300)
model_1_300.summary()

# Entrenar el modelo 1 con los embeddings de 300 dimensiones
model_1_300.fit(x_train_pad, y_train_encoded, epochs=5, batch_size=32, validation_data=(x_test_pad, y_test_encoded))

# Evaluar el modelo 1 con los embeddings de 300 dimensiones
accuracy, precision, recall = evaluate_model(model_1_300, x_test_pad, y_test_encoded)
print(f"Modelo 1 con 300 dimensiones - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 55, 300)           7055400   
                                                                 
 flatten_2 (Flatten)         (None, 16500)             0         
                                                                 
 dense_4 (Dense)             (None, 128)               2112128   
                                                                 
 dense_5 (Dense)             (None, 3)                 387       
                                                                 
Total params: 9,167,915
Trainable params: 2,112,515
Non-trainable params: 7,055,400
_________________________________________________________________
Epoch 1/5
374/374 [==============================] - 7s 16ms/step - loss: 0.4693 - accuracy: 0.8038 - val_loss: 0.3893 - val_accuracy: 0.8417
Epoch 2/5
374/374 [=======

### Modelo 2

In [18]:
# Instanciar el modelo 2 con los embeddings de 50 dimensiones
model_2_50 = create_ffnn_model_2(embedding_layer_50)
model_2_50.summary()

# Entrenar el modelo 2 con los embeddings de 50 dimensiones
model_2_50.fit(x_train_pad, y_train_encoded, epochs=5, batch_size=32, validation_data=(x_test_pad, y_test_encoded))

# Evaluar el modelo 2 con los embeddings de 50 dimensiones
accuracy, precision, recall = evaluate_model(model_2_50, x_test_pad, y_test_encoded)
print(f"Modelo 2 con 50 dimensiones - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 55, 50)            1175900   
                                                                 
 flatten_3 (Flatten)         (None, 2750)              0         
                                                                 
 dense_6 (Dense)             (None, 256)               704256    
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dense_8 (Dense)             (None, 3)                 387       
                                                                 
Total params: 1,913,439
Trainable params: 737,539
Non-trainable params: 1,175,900
_________________________________________________________________
Epoch 1/5
374/374 [===================

In [19]:
# Instanciar el modelo 2 con los embeddings de 100 dimensiones
model_2_100 = create_ffnn_model_2(embedding_layer_100)
model_2_100.summary()

# Entrenar el modelo 2 con los embeddings de 100 dimensiones
model_2_100.fit(x_train_pad, y_train_encoded, epochs=5, batch_size=32, validation_data=(x_test_pad, y_test_encoded))

# Evaluar el modelo 2 con los embeddings de 100 dimensiones
accuracy, precision, recall = evaluate_model(model_2_100, x_test_pad, y_test_encoded)
print(f"Modelo 2 con 100 dimensiones - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 55, 100)           2351800   
                                                                 
 flatten_4 (Flatten)         (None, 5500)              0         
                                                                 
 dense_9 (Dense)             (None, 256)               1408256   
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 dense_11 (Dense)            (None, 3)                 387       
                                                                 
Total params: 3,793,339
Trainable params: 1,441,539
Non-trainable params: 2,351,800
_________________________________________________________________
Epoch 1/5
374/374 [=================

In [20]:
# Instanciar el modelo 2 con los embeddings de 300 dimensiones
model_2_300 = create_ffnn_model_2(embedding_layer_300)
model_2_300.summary()

# Entrenar el modelo 2 con los embeddings de 300 dimensiones
model_2_300.fit(x_train_pad, y_train_encoded, epochs=5, batch_size=32, validation_data=(x_test_pad, y_test_encoded))

# Evaluar el modelo 2 con los embeddings de 300 dimensiones
accuracy, precision, recall = evaluate_model(model_2_300, x_test_pad, y_test_encoded)
print(f"Modelo 2 con 300 dimensiones - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 55, 300)           7055400   
                                                                 
 flatten_5 (Flatten)         (None, 16500)             0         
                                                                 
 dense_12 (Dense)            (None, 256)               4224256   
                                                                 
 dense_13 (Dense)            (None, 128)               32896     
                                                                 
 dense_14 (Dense)            (None, 3)                 387       
                                                                 
Total params: 11,312,939
Trainable params: 4,257,539
Non-trainable params: 7,055,400
_________________________________________________________________
Epoch 1/5
374/374 [================

### Modelo 3

In [21]:
# Instanciar el modelo 3 con los embeddings de 50 dimensiones
model_3_50 = create_ffnn_model_3(embedding_layer_50)
model_3_50.summary()

# Entrenar el modelo 3 con los embeddings de 50 dimensiones
model_3_50.fit(x_train_pad, y_train_encoded, epochs=5, batch_size=32, validation_data=(x_test_pad, y_test_encoded))

# Evaluar el modelo 3 con los embeddings de 50 dimensiones
accuracy, precision, recall = evaluate_model(model_3_50, x_test_pad, y_test_encoded)
print(f"Modelo 3 con 50 dimensiones - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 55, 50)            1175900   
                                                                 
 flatten_6 (Flatten)         (None, 2750)              0         
                                                                 
 dense_15 (Dense)            (None, 512)               1408512   
                                                                 
 dense_16 (Dense)            (None, 256)               131328    
                                                                 
 dense_17 (Dense)            (None, 128)               32896     
                                                                 
 dense_18 (Dense)            (None, 3)                 387       
                                                                 
Total params: 2,749,023
Trainable params: 1,573,123
No

In [22]:
# Instanciar el modelo 3 con los embeddings de 100 dimensiones
model_3_100 = create_ffnn_model_3(embedding_layer_100)
model_3_100.summary()

# Entrenar el modelo 3 con los embeddings de 100 dimensiones
model_3_100.fit(x_train_pad, y_train_encoded, epochs=5, batch_size=32, validation_data=(x_test_pad, y_test_encoded))

# Evaluar el modelo 3 con los embeddings de 100 dimensiones
accuracy, precision, recall = evaluate_model(model_3_100, x_test_pad, y_test_encoded)
print(f"Modelo 3 con 100 dimensiones - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 55, 100)           2351800   
                                                                 
 flatten_7 (Flatten)         (None, 5500)              0         
                                                                 
 dense_19 (Dense)            (None, 512)               2816512   
                                                                 
 dense_20 (Dense)            (None, 256)               131328    
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dense_22 (Dense)            (None, 3)                 387       
                                                                 
Total params: 5,332,923
Trainable params: 2,981,123
No

In [23]:
# Instanciar el modelo 3 con los embeddings de 300 dimensiones
model_3_300 = create_ffnn_model_3(embedding_layer_300)
model_3_300.summary()

# Entrenar el modelo 3 con los embeddings de 300 dimensiones
model_3_300.fit(x_train_pad, y_train_encoded, epochs=5, batch_size=32, validation_data=(x_test_pad, y_test_encoded))

# Evaluar el modelo 3 con los embeddings de 300 dimensiones
accuracy, precision, recall = evaluate_model(model_3_300, x_test_pad, y_test_encoded)
print(f"Modelo 3 con 300 dimensiones - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 55, 300)           7055400   
                                                                 
 flatten_8 (Flatten)         (None, 16500)             0         
                                                                 
 dense_23 (Dense)            (None, 512)               8448512   
                                                                 
 dense_24 (Dense)            (None, 256)               131328    
                                                                 
 dense_25 (Dense)            (None, 128)               32896     
                                                                 
 dense_26 (Dense)            (None, 3)                 387       
                                                                 
Total params: 15,668,523
Trainable params: 8,613,123
N